In [ ]:
import textblob   
import pandas as pd
import csv
import time
import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint
import numpy as np     

import matplotlib.pyplot as plt
headers = {"Accept-Language": "en-US, en;q=0.5"}

Base_url = "https://uk.reuters.com"

#from reuters oil archive
pages = [str(i) for i in range(1,2000)]
url_list=[]

for page in pages:

    url= 'https://uk.reuters.com/news/archive/oilRpt?view=page&page='+page +'&pageSize=10'
    url_list.append(url)

print(url_list)

In [ ]:
from warnings import warn



In [ ]:
from IPython.core.display import clear_output
from time import time

start_time = time()
requests = 0

for _ in range(5):
    
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

In [ ]:
# Create an empty list which will contain the selected news articles 
List_of_links = [] 
word1="article"

for newsurl in url_list:
    response = get(newsurl, headers = headers)
    
    sleep(randint(1,3))
    
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
    
    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
    
    # Parse the content of the request with BeautifulSoup
    #page_html = BeautifulSoup(response.text, 'html.parser')
    html_soup = BeautifulSoup(response.text, 'html.parser')
    sub_links = html_soup.find_all('a')
    for links in sub_links:
        sp = BeautifulSoup(str(links),'html.parser')  # first convert into a string
        tag = sp.a
      #if word1 in tag['title'] or word2 in tag['title'] or word3 in tag['title']:
        if word1 in tag['href']:
            category_links =  Base_url + tag["href"]
            List_of_links.append(category_links)
            
    
    

In [ ]:
unique_links = list(set(List_of_links))
for q in unique_links: print(q)


In [ ]:
df = pd.DataFrame({
                   'url_news'  : unique_links,
                  })

print(df.info())
df

In [ ]:
df.to_csv('links.csv')

In [ ]:
#read in links

dflinks=pd.read_csv('C:/Users/neil.watt/Documents/PythonScripts/Sentiment/Main/MarketSentiment_Analysis/links.csv')
dflinks['url_news']


In [ ]:


unique_links=dflinks['url_news']
# Create an empty list which will contain the selected news articles 
text_body=[]
date=[]


for selected_links in unique_links:
    response = get(selected_links, headers = headers)
    sleep(randint(1,3))
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
      # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        
    html_soup = BeautifulSoup(response.text, 'html.parser')
    
    #retrieve body of article
    results_text = html_soup.find(class_='StandardArticleBody_body')
    story_text = results_text.get_text()
    lower_story_text=story_text.lower()
    text_body.append(lower_story_text) 
    #retrieve date
    date_results_text = html_soup.find(class_='ArticleHeader_date')
    date_text = date_results_text.get_text()
    date.append(date_text) 
    
    
    
    


In [ ]:
df_articles = pd.DataFrame({
                   'text_body'  : text_body, 'date' : date
                  })

print(df_articles.info())
df_articles

In [ ]:
df_articles.to_csv('df_articles.csv')

In [ ]:
dfarticles=pd.read_csv('C:/Users/neil.watt/Documents/PythonScripts/Sentiment/Main/MarketSentiment_Analysis/df_articles.csv')


In [ ]:
from textblob import TextBlob
import re

#first I want to compare textblob inbuild sentiment analysis
#here defining functions to clean the text

def clean_text(text):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())


#analyze_text
def analyze_text(text):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    
    #The bottom 25% of sentiment is -0.017
    #The top 25% of sentiment is 0.08
    
    analysis = TextBlob(clean_text(text))
    if analysis.sentiment.polarity > 0.08:
        return 1
    elif analysis.sentiment.polarity < -0.017:
        return -1
    else:
        return 0
    
   # elif analysis.sentiment.polarity == 0:
   #     return 0
    #else:
    #    return -1
    
def analyze_text_polarity(text):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_text(text))
    return analysis.sentiment.polarity
    
    


In [ ]:
dfoil=pd.read_csv('dict.csv')

positive_words=dfoil[dfoil["Sentiment"]=="positive"]['Word'].values
negative_words=dfoil[dfoil["Sentiment"]=="negative"]['Word'].values
print(negative_words[-10:])

In [ ]:

import re

def tokenizer(theText):
    theTokens = re.findall(r'\b\w[\w-]*\b', theText)
    return theTokens


    
def calculator(text):
    # Count positive words
    numPosWords = 0
    cleantext = tokenizer((text))
    #cleantext = clean_text(text)
    for word in cleantext:
        if word in positive_words:
            numPosWords += 1
            
    # Count negative words
    numNegWords = 0
    for word in cleantext:
        if word in negative_words:
            numNegWords += 1
    
    sum = (numPosWords - numNegWords)
    return sum




In [ ]:
# Compute sentiment score given dictionary
dfarticles['dict_sentiment'] = np.array([calculator(text) for text in dfarticles['text_body'] ])
dfarticles['dict_sentiment']

In [ ]:
# Compute sentiment score given textblob sentiment polarity
dfarticles['sentiment_polarity'] = np.array([ analyze_text_polarity(text) for text in dfarticles['text_body'] ])
dfarticles['sentiment_polarity']

In [ ]:
dfarticles['sentiment_polarity'].describe()

In [ ]:
# Compute sentiment score given textblob sentiment (binary)
dfarticles['sentiment'] = np.array([ analyze_text(text) for text in dfarticles['text_body'] ])


In [ ]:
dfarticles.to_csv('sentiment_analysis.csv')

In [ ]:
dfsentiment=pd.read_csv('C:/Users/neil.watt/Documents/PythonScripts/Sentiment/Main/MarketSentiment_Analysis/sentiment_analysis.csv')
dfsentiment

In [ ]:
df_daily.to_csv('mean_scores.csv')

In [ ]:
dfbrent=pd.read_csv('C:/Users/neil.watt/Documents/PythonScripts/Sentiment/Main/MarketSentiment_Analysis/brent.csv')
#dfbrent['Date']
dfbrent["Date"]=dfbrent["Date"].astype('datetime64[ns]') # Convert column to date format


dfbrent['10_MA_sentiment'] = dfbrent['sentiment'].rolling(window=10).mean()
dfbrent['30_MA_sentiment'] = dfbrent['sentiment'].rolling(window=30).mean()
dfbrent2=dfbrent.dropna()


In [ ]:
#0.26 correlatin between 10 day MA sentiment
np.corrcoef(dfbrent2['10_MA_sentiment'], dfbrent2["Brent"])



In [ ]:
#0.4 correlation between 30 day MA sentiment
np.corrcoef(dfbrent2['30_MA_sentiment'], dfbrent2["Brent"])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()


ax1.plot(dfbrent['Date'].astype('O'),  dfbrent2["Brent"], color='black', linewidth=3.3)
ax1.set_xlabel('')
# Make the y-axis label, ticks and tick labels match the line color.

ax1.set_ylabel('Brent', color='black', fontsize=20)
ax1.tick_params('y', colors='black')

ax2 = ax1.twinx()
ax1.xaxis.set_major_locator(dates.MonthLocator(interval=1))

ax2.plot(dfbrent['Date'].astype('O'),  dfbrent2['10_MA_sentiment'],'green', linewidth=1.3)
ax2.plot(dfbrent['Date'].astype('O'),  dfbrent2['30_MA_sentiment'],'red', linewidth=3.3)
ax2.set_ylabel('Sentiment', color='black', fontsize=20)
ax2.tick_params('y', colors='black')
fig.set_size_inches(18.5, 10.5)
fig.tight_layout()
plt.title('Brent versus Sentiment', fontsize=20)


#add legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.show()